# Création de nouvelles ressources

Initialisation de Vortex

In [1]:
%load_ext ivortex
%vortex
toolbox.active_verbose = False
toolbox.active_now = False

Vortex 1.6.0 loaded ( Thursday 24. January 2019, at 15:41:56 )


## Création d'une ressource pour un usage via le *provider* *Vortex*

Cette ressource doit :

* Etre décrite par des attributs *kind*, *model*, *date*, *cutoff*, *number* et *scope*
* L'attribut *scope* ne pourra prendre comme valeurs que ``surface`` ou ``altitude``

Plusieurs versions de cette nouvelle ressource seront créées pour les besoins du cours (afin de décrire les différentes manières de procéder) : dans la réalité, il ne faudrait créer qu'une seule classe.

### Modèle de programation "classique"

In [2]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco

class DemoResource1(FlowResource):

    _footprint = [
        number_deco,
        dict(info = 'Description 1',
             attr = dict(
                kind = dict(values = ['demo1']),
                scope = dict(info = 'Scope description',
                             values = ['surface', 'altitude'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'
    
    def namebuilding_info(self):
        """On ajoute l'attribut *scope*."""
        bdict = super(DemoResource1, self).namebuilding_info()
        bdict['src'].append(self.scope)
        return bdict

Ici, la méthode **namebuilding_info** est redéfinie manuellement. En revanche, on tire bien profit de l'héritage (**FlowResource** fournis les attributs *model*, *date* et *cutoff*) et des attributs footprints pré-définis (``number_deco`` fourni l'attribut *number*)

Testons le résultat :

In [3]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo1', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

### Utilisation d'un décorateur

In [4]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
from vortex.syntax.stddeco import namebuilding_append

@namebuilding_append('src', lambda self: self.scope)
class DemoResource2(FlowResource):

    _footprint = [
        number_deco,
        dict(info = 'Description 2',
             attr = dict(
                kind = dict(values = ['demo2']),
                scope = dict(info = 'Scope description',
                             values = ['surface', 'altitude'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'

Ici, c'est le décorateur de classe ``namebuilding_append`` qui se charge d'ajouter l'attribut *scope* dans les entrées du dictionnaire renvoyé par **namebuilding_info**...

Testons le résultat :

In [5]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo2', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

### Création d'un attribut pré-défini

In [6]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
from vortex.syntax.stddeco import namebuilding_append

demoscope = fp.Footprint(info = 'Abstract Scope',
                         attr = dict(scope = dict(info = 'Scope description',
                                                 values = ['surface', 'altitude'])))

demoscope_deco = fp.DecorativeFootprint(
    demoscope,
    decorator = [namebuilding_append('src', lambda self: self.scope), ])

class DemoResource3(FlowResource):

    _footprint = [
        number_deco,
        demoscope_deco,
        dict(info = 'Description 3',
             attr = dict(kind = dict(values = ['demo3'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'

On défini un attribut prédéfini pour *scope* (objet ``demoscope``) et on l'associe à un décorateur de classe (objet ``demoscope_deco``). Si l'attribut, *scope* a des chances d'être utilisé dans d'autres ressources, cette stratégie peut être rentable.

Testons le résultat :

In [7]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo3', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

# Extension de l'exemple précédent pour le *provider* Olive

In [8]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
# demoscope_deco (défini plus haut est ici réutilisé)

class DemoResource4(FlowResource):

    _footprint = [
        number_deco,
        demoscope_deco,
        dict(info = 'Description 4',
             attr = dict(kind = dict(values = ['demo4'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'
    
    def olive_basename(self):
        return '{0.realkind:s}{0.scope:s}{0.number:04d}'.format(self).upper()

Testons le résultat :

In [9]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo4', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='olive.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'olive://olive.cache.fr/ABCD/20190101H00A/anyblock/DEMOALTITUDE0005'

Cela fonctionne encore avec Vortex...

In [10]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo4', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

# Définition d'une constante (récupérable via genv)

La ressource en question sera référencée dans Genv par la clé 'DEMOFILE' 

In [11]:
from common.data.consts import GenvModelResource

class DemoConstante1(GenvModelResource):

    _footprint = dict(info = 'Constante 1',
                      attr = dict(kind = dict(values = ['ma_constante1']),
                                  gvar = dict(default = 'DEMOFILE')))
    
    @property
    def realkind(self):
        return 'democonstante'

Préparons le test :

In [12]:
# Dans la vraie vie, cela sera fait automatique par Olive ou le système de création de jobs
from gco.tools import genv
genv.register(cycle='test_cycle.01',
              DEMOFILE='the_test_constant.01')

Le test à proprement parler :

In [13]:
rtest = toolbox.input(model='arpege', kind='ma_constante1',
                      # Provider + Container :
                      genv = 'test_cycle.01', incore=True)
rtest[0].location()

'gget://gco.meteo.fr/tampon/the_test_constant.01'

## Et si cette constante est mensuelle ?

ou, le retour de l'attribut prédéfini...

In [14]:
from vortex.syntax.stdattrs import month_deco
from common.data.consts import GenvModelResource

class DemoConstante2(GenvModelResource):

    _footprint = [month_deco,
                  dict(info = 'Constante 2',
                       attr = dict(kind = dict(values = ['ma_constante2']),
                                   gvar = dict(default = 'DEMOFILE')))
                 ]
    
    @property
    def realkind(self):
        return 'democonstante'

Testons le résultat :

In [15]:
rtest = toolbox.input(model='arpege', kind='ma_constante2', month=3,
                      # Provider + Container :
                      genv = 'test_cycle.01', incore=True)
rtest[0].location()

'gget://gco.meteo.fr/tampon/the_test_constant.01.m03'

Si l'on va regarder la définition de l'attribut pré-défini *month_deco*, on s'aperçoit que celui-ci est associé à un décorateur de classe qui altère le contenu de la méthode **gget_basename** : c'est ce qui explique l'apparition du '.m03' dans l'URI.